# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [50]:
# import libraries
from sqlalchemy import create_engine

import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', con = engine)

predict_cols = ['request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']
X = df['message']
y = df[predict_cols]
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [53]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemm = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemm.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [54]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    #('clf', MultiOutputClassifier(RandomForestClassifier()))
    #('clf', MultiOutputClassifier(SVC()))
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [56]:
y_train.sum()

request                   2944.0
offer                       72.0
aid_related               7224.0
medical_help              1369.0
medical_products           875.0
search_and_rescue          478.0
security                   309.0
military                   582.0
child_alone                  0.0
water                     1101.0
food                      1917.0
shelter                   1543.0
clothing                   259.0
money                      393.0
missing_people             201.0
refugees                   570.0
death                      789.0
other_aid                 2324.0
infrastructure_related    1161.0
transport                  809.0
buildings                  876.0
electricity                350.0
tools                      103.0
hospitals                  187.0
shops                       75.0
aid_centers                210.0
other_infrastructure       789.0
weather_related           4849.0
floods                    1424.0
storm                     1644.0
fire      

In [57]:
# some classifiers cannot handle categories with 0 entries
# enable if using SVC
if False:
    y_train.drop(columns = ['child_alone'], inplace = True)
    y_test.drop(columns = ['child_alone'], inplace = True)
    # will also have to remove item from predict_cols otherwise classification_report will fail

In [58]:
y_train.shape

(17448, 35)

In [59]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [60]:
y_preds = pipeline.predict(X_test)

In [61]:
print(classification_report(y_test, y_preds, labels=predict_cols))

             precision    recall  f1-score   support

          0       0.43      0.12      0.19      1495
          1       0.00      0.00      0.00        46
          2       0.46      0.14      0.22      3563
          3       0.00      0.00      0.00       702
          4       0.00      0.00      0.00       425
          5       0.00      0.00      0.00       245
          6       0.00      0.00      0.00       160
          7       0.00      0.00      0.00       272
          8       0.00      0.00      0.00         0
          9       0.00      0.00      0.00       555
         10       0.13      0.01      0.01       989
         11       0.07      0.00      0.00       752
         12       0.17      0.02      0.04       140
         13       0.00      0.00      0.00       208
         14       0.00      0.00      0.00        97
         15       0.00      0.00      0.00       299
         16       0.00      0.00      0.00       397
         17       0.12      0.00      0.01   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [62]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f91fc371048>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [63]:
parameters = {'vect__stop_words':[None, set(nltk.corpus.stopwords.words('english'))],
              # RandomForestClassifier
             #'clf__estimator__n_estimators':[10, 20, 50, 100],
             #'clf__estimator__criterion':['gini', 'entropy']
             'clf__estimator__learning_rate': [0.5, 0.8, 1, 1.2]
             }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)
cv.fit(X_train, y_train)
y_preds = cv.predict(X_test)
print(classification_report(y_test, y_preds))
# RandomForestClassifier was .34 .07 .10
# SVC was unable to predict anything in multioutput, but a single classifier got .29 acc

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__learning_rate=0.5, vect__stop_words=None ........
[CV]  clf__estimator__learning_rate=0.5, vect__stop_words=None, score=0.38995873452544705, total= 1.6min
[CV] clf__estimator__learning_rate=0.5, vect__stop_words=None ........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.8min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.5, vect__stop_words=None, score=0.3786107290233838, total= 1.6min
[CV] clf__estimator__learning_rate=0.5, vect__stop_words=None ........


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.6min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.5, vect__stop_words=None, score=0.39322558459422285, total= 1.6min
[CV] clf__estimator__learning_rate=0.5, vect__stop_words={"don't", 'was', 'very', "haven't", 'no', 'ma', 'how', 'myself', "shan't", 'weren', 'won', 'herself', "hadn't", 'him', "you'd", 'haven', 'isn', 'he', 'ourselves', 'are', 'after', 'being', "wasn't", 'his', 'who', 'her', 'll', 'here', 'before', 'so', 'their', 'when', 'if', 'further', 'whom', 'it', 'will', 'into', "couldn't", 'at', 'shouldn', 'she', "doesn't", 'now', 'my', 'mightn', 'didn', "mustn't", 'each', 'than', 'don', 'wouldn', "it's", 'been', "needn't", 'had', 'those', 'until', 'which', 'that', "that'll", 'up', 'through', 'or', 'between', 'the', "aren't", 'during', "isn't", 'm', 'its', 'more', "weren't", 'some', 'with', 'an', 'most', 'themselves', 'y', 'ain', 'me', 'from', 'hers', 'what', 'again', 'both', 'about', "she's", 'yours', "hasn't", 'have', 'as', 'then', 'there', 'your', 'only', 'same', 'mustn', 'doing', 'needn', 

...

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 14.8min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.8, vect__stop_words={"don't", 'was', 'very', "haven't", 'no', 'ma', 'how', 'myself', "shan't", 'weren', 'won', 'herself', "hadn't", 'him', "you'd", 'haven', 'isn', 'he', 'ourselves', 'are', 'after', 'being', "wasn't", 'his', 'who', 'her', 'll', 'here', 'before', 'so', 'their', 'when', 'if', 'further', 'whom', 'it', 'will', 'into', "couldn't", 'at', 'shouldn', 'she', "doesn't", 'now', 'my', 'mightn', 'didn', "mustn't", 'each', 'than', 'don', 'wouldn', "it's", 'been', "needn't", 'had', 'those', 'until', 'which', 'that', "that'll", 'up', 'through', 'or', 'between', 'the', "aren't", 'during', "isn't", 'm', 'its', 'more', "weren't", 'some', 'with', 'an', 'most', 'themselves', 'y', 'ain', 'me', 'from', 'hers', 'what', 'again', 'both', 'about', "she's", 'yours', "hasn't", 'have', 'as', 'then', 'there', 'your', 'only', 'same', 'mustn', 'doing', 'needn', 'above', 'wasn', 'itself', 'for', 'under', 'o', "shouldn't", 'down', "won't", 'couldn', 'did', 'other', 

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 37.7min finished


             precision    recall  f1-score   support

          0       0.50      0.07      0.13      1495
          1       0.00      0.00      0.00        46
          2       0.49      0.06      0.11      3563
          3       0.00      0.00      0.00       702
          4       0.00      0.00      0.00       425
          5       0.00      0.00      0.00       245
          6       0.00      0.00      0.00       160
          7       0.00      0.00      0.00       272
          8       0.00      0.00      0.00         0
          9       0.00      0.00      0.00       555
         10       0.00      0.00      0.00       989
         11       0.00      0.00      0.00       752
         12       0.00      0.00      0.00       140
         13       0.00      0.00      0.00       208
         14       0.00      0.00      0.00        97
         15       0.00      0.00      0.00       299
         16       0.00      0.00      0.00       397
         17       0.17      0.00      0.00   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:

filename = 'finalized_model.sav'
pickle.dump(cv.best_estimator_, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.